In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams 
from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import xgboost as xgb

eng_stopwords = set(stopwords.words('english'))

from pandas import DataFrame

# *Prepare data*

In [30]:
train1 = pd.read_csv("../train/train1.csv")
test1 = pd.read_csv("../test/test1.csv")
print("train1:",train1.shape)
print("test1:",test1.shape)

train2 = pd.read_csv("../train/train2.csv")
test2 = pd.read_csv("../test/test2.csv")
print("train2:",train2.shape)
print("test2:",test2.shape)

train2.head()

train1: (404290, 8)
test1: (2345796, 5)
train2: (404290, 7)
test2: (2345796, 7)


,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,id
0,6.0,0.857143,5.0,0.625000,4.0,0.571429,0
1,6.0,0.461538,4.0,0.307692,3.0,0.250000,1
2,3.0,0.300000,0.0,0.000000,0.0,0.000000,2
3,1.0,0.062500,0.0,0.000000,0.0,0.000000,3
4,3.0,0.200000,0.0,0.000000,0.0,0.000000,4


In [11]:
train1.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000


In [10]:
test1.head()

,test_id,question1,question2,word_match,tfidf_word_match
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,0.266667,0.274019
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,0.500000,0.480962
2,2,What but is the best way to send money from Ch...,What you send money to China?,0.444444,0.468893
3,3,Which food not emulsifiers?,What foods fibre?,0.000000,0.000000
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,0.800000,1.000000


In [5]:
train1['xxx']=train1.qid1-train1.qid2
train1.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,xxx
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164,-1
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758,-1
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191,-1
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000,-1
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000,-1


In [6]:
train1[(train1["xxx"]!=-1)]#184930 rows × 9 columns

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,xxx
264,264,81,529,Why do Slavs squat?,Do squats work for men?,0,0.000000,0.000000,-448
326,326,652,108,What are the most interesting foods you have e...,What is the most delicious dish you've ever ea...,1,0.200000,0.233642,544
353,353,705,134,What are the most important books ever written?,What is the most important book you have ever ...,1,0.500000,0.433054,571
647,647,1292,1127,How do I make money with YouTube?,How do I make money through YouTube?,1,1.000000,1.000000,165
704,704,1189,1405,When will the best time for having sex?,What would be the best time to have sex?,1,0.857143,0.909044,-216
729,729,883,1454,Why is Saltwater taffy candy imported in China?,Why is Saltwater taffy candy imported in Laos?,1,0.800000,0.799245,-571
1027,1027,2049,1919,What food should I eat to gain weight?,How could I gain weight in a healthy way?,0,0.222222,0.236910,130
1081,1081,1863,2156,Did Trump win the election?,What do you think of Trump winning the elections?,0,0.285714,0.219689,-293
1114,1114,2221,1093,How can I hack a WhatsApp account?,How can I hack another WhatsApp account?,1,0.857143,0.855327,1128
1332,1332,2656,1215,Is a third world war coming?,Is World War 3 more imminent than expected?,1,0.444444,0.365184,1441


In [32]:
#train2.drop(['question1','question2'],axis=1, inplace=True)
#test2.drop(['question1','question2'],axis=1, inplace=True)

train_df.drop(['qid1','qid2','question1','question2'],axis=1, inplace=True)
test_df.drop(['question1','question2'],axis=1, inplace=True)

In [31]:
train_df=pd.merge(train1,train2,how='inner', on='id') 
test_df=pd.merge(test1,test2,how='inner', on='test_id') 

del train1,train2,test1,test2

print("train_df:",train_df.shape)
print("test_df",test_df.shape)
train_df.head()

train_df: (404290, 14)
test_df (2345796, 11)


,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164,6.0,0.857143,5.0,0.625000,4.0,0.571429
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758,6.0,0.461538,4.0,0.307692,3.0,0.250000
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191,3.0,0.300000,0.0,0.000000,0.0,0.000000
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000,1.0,0.062500,0.0,0.000000,0.0,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000,3.0,0.200000,0.0,0.000000,0.0,0.000000


In [33]:
print("train_df:",train_df.shape)
print("test_df",test_df.shape)
test_df.head()

train_df: (404290, 10)
test_df (2345796, 9)


,test_id,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio
0,0,0.266667,0.274019,4.0,0.333333,2.0,0.142857,1.0,0.076923
1,1,0.500000,0.480962,5.0,0.555556,1.0,0.083333,0.0,0.000000
2,2,0.444444,0.468893,4.0,0.571429,2.0,0.285714,1.0,0.166667
3,3,0.000000,0.000000,1.0,0.200000,0.0,0.000000,0.0,0.000000
4,4,0.800000,1.000000,3.0,0.500000,2.0,0.400000,1.0,0.250000


# *Add features to the Data*

# *Rebalancing the Data*

In [15]:
y_train = train_df['is_duplicate'].values
pos_train = train_df[y_train == 1]
neg_train = train_df[y_train == 0]


# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

x_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train

print("train_df:",train_df.shape)
print("test_df",test_df.shape)

print("x_train:",x_train.shape)
print("y_train",len(y_train))

0.19124366100096607
train_df: (404290, 10)
test_df (2345796, 9)
x_train: (780486, 10)
y_train 780486


In [34]:
#x_train.drop(['id'],axis=1, inplace=True)
sub = pd.DataFrame()
sub['test_id'] = test_df['test_id']
x_test=test_df.drop(['test_id'],axis=1)

#sub['is_duplicate'] = p_test

# XGBoost

In [38]:
# Finally, we split some of the data off for validation
from sklearn.cross_validation import train_test_split
#x_train.drop(['id','is_duplicate'],axis=1, inplace=True)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)
print("x_train:",x_train.shape)
print("y_train",len(y_train))
print("x_valid:",x_valid.shape)
print("y_valid",len(y_valid))

print("x_test:",x_test.shape)

x_train: (163678, 8)
y_train 163678
x_valid: (40920, 8)
y_valid 40920
x_test: (2345796, 8)


In [40]:
import xgboost as xgb

# Set our parameters for xgboost
#params = {}
#params['objective'] = 'binary:logistic'
#params['eval_metric'] = 'logloss'
#params['eta'] = 0.02
#params['max_depth'] = 6

#XGBoost自带接口
params={
    'eta': 0.1,
    'max_depth':6,   
    'min_child_weight':1,
    'gamma':0.3, 
    'subsample':0.8,
    'colsample_bytree':0.8,
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'nthread':12,
    'scale_pos_weight': 1,
    'lambda':1,  
    'seed':27,
    'silent':0 ,
    'eval_metric': 'logloss'
}

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(x_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 3000, watchlist, early_stopping_rounds=500, verbose_eval=10)

[0]	train-logloss:0.669361	valid-logloss:0.669503
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 500 rounds.
[10]	train-logloss:0.511365	valid-logloss:0.51186
[20]	train-logloss:0.439359	valid-logloss:0.440353
[30]	train-logloss:0.400306	valid-logloss:0.401627
[40]	train-logloss:0.378652	valid-logloss:0.380335
[50]	train-logloss:0.366316	valid-logloss:0.36834
[60]	train-logloss:0.359384	valid-logloss:0.361792
[70]	train-logloss:0.354689	valid-logloss:0.357411
[80]	train-logloss:0.351856	valid-logloss:0.354851
[90]	train-logloss:0.350063	valid-logloss:0.353261
[100]	train-logloss:0.348823	valid-logloss:0.352279
[110]	train-logloss:0.347803	valid-logloss:0.351468
[120]	train-logloss:0.347021	valid-logloss:0.350868
[130]	train-logloss:0.346393	valid-logloss:0.35043
[140]	train-logloss:0.345662	valid-logloss:0.349957
[150]	train-logloss:0.345048	valid-logloss:0.349563
[160]	train-logloss:0.344396	v

In [42]:
y_bst= bst.predict(d_test)
sub['is_duplicate'] = y_bst
sub

,test_id,is_duplicate
0,0,0.043764
1,1,0.382057
2,2,0.331392
3,3,0.012511
4,4,0.007249
5,5,0.079624
6,6,0.371148
7,7,0.473786
8,8,0.385476
9,9,0.001712


In [43]:
sub.to_csv('../output/xgb_20170419_A.csv', index=False)

# Explore

In [25]:
def word1(que):
    return [word for word in word_tokenize(que.lower()) if word not in eng_stopwords]#word_tokenize:对句子进行分词

def word2(que):
    return [word for word in que.lower().split() if word not in eng_stopwords]#:对句子进行分词

def get_bigrams(que):
    return [i for i in ngrams(word_tokenize(que.lower()), 2)]#组合分词

In [26]:
sentence1 = """At eight o'clock on Thursday morning Arthur didn't feel very good?"""
sentence2 = """At eight o'clock on Thursday morning Arthur feel very good!"""
x1= word1(sentence1)
x2= word1(sentence2)

y1= word2(sentence1)
y2= word2(sentence2)

z1= get_bigrams(sentence1)
z2= get_bigrams(sentence2)

print(x1)
print(x2)
print("  ")
print(y1)
print(y2)
print("  ")
print(z1)
print(z2)


['eight', "o'clock", 'thursday', 'morning', 'arthur', "n't", 'feel', 'good', '?']
['eight', "o'clock", 'thursday', 'morning', 'arthur', 'feel', 'good', '!']
  
['eight', "o'clock", 'thursday', 'morning', 'arthur', "didn't", 'feel', 'good?']
['eight', "o'clock", 'thursday', 'morning', 'arthur', 'feel', 'good!']
  
[('at', 'eight'), ('eight', "o'clock"), ("o'clock", 'on'), ('on', 'thursday'), ('thursday', 'morning'), ('morning', 'arthur'), ('arthur', 'did'), ('did', "n't"), ("n't", 'feel'), ('feel', 'very'), ('very', 'good'), ('good', '?')]
[('at', 'eight'), ('eight', "o'clock"), ("o'clock", 'on'), ('on', 'thursday'), ('thursday', 'morning'), ('morning', 'arthur'), ('arthur', 'feel'), ('feel', 'very'), ('very', 'good'), ('good', '!')]
